___

programmer: `@crispengari`

main: `Linear Regression`

description: `Given features as a dense vector we want to predict insuarance value. This is a basic regression task using pytorch.`

language: `python`
___

### Insuarance Prediction

Given the following features:

1. age	
2. sex	
3. bmi	
4. children	
5. smoker	
6. region

We want to predict the `insuarance` charges.

### Basic Imports

In the following code cell we are going to import the basic packages that we are going to use throughout this notebook.

In [4]:
import torch
import numpy as np
import pandas as pd
import random
import os

from sklearn.model_selection import train_test_split
from torch import nn
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, Normalizer

torch.__version__

'1.9.1+cpu'

### SEED

In the following code cell we are going to set the `SEED` for reproducivity. Our seed value will be a constant value of `42`.

In [5]:
SEED = 42

### Device
In the following code cell we are going to set a device `gpu` to enable gpu accellaration if possible.

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Data
Our data we will be coming from a csv file called `insurance.csv` and we will read it into a dataframe using the pandas package.

In [7]:
df = pd.read_csv('../insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Features and Labels

Some of our features are not numeric for example in the `sex` column. We need to convert these features to numeric. Luckly sklearn has a build in class called `LabelEncoder` from preprocessing that allows us to change these features that are not numeric to numeric. In the following code cell we will do that task. You can read more about them [here](https://scikit-learn.org/stable/modules/preprocessing.html).

In [9]:
sex_encoder = LabelEncoder().fit(df.sex)
sex = sex_encoder.transform(df.sex)

smoking_encoder = LabelEncoder().fit(df.smoker)
smoking = smoking_encoder.transform(df.smoker)

region_encoder = LabelEncoder().fit(df.region)
region = region_encoder.transform(df.region)

In [12]:
sex_encoder.transform(['male', 'female'])

array([1, 0], dtype=int64)

### Extracting features and labels.

In [13]:
features = df.drop(["charges"], axis=1)
labels = df.iloc[:, -1].values

In [14]:
features.head(1)

,age,sex,bmi,children,smoker,region
0,19,female,27.9,0,yes,southwest


In [15]:
labels[0]

16884.924

### Changing the features columns to numeric

In the following code cell we are going to change the features that are not numeric to numeric by assigning the encoded and transformed values to each column.

In [16]:
features['sex'] = sex
features['smoker'] = smoking
features['region'] = region

In [17]:
features.head(1)

,age,sex,bmi,children,smoker,region
0,19,0,27.9,0,1,3


### Features transformation

In the following code cell we are going to tranform our features so that it will be easy to train our model.

In [19]:
column_transformer = ColumnTransformer([
    ('min_max_scaler', MinMaxScaler(), ["age", 'children']),
    ('standard_scaler', StandardScaler(), ['smoker', 'region']),
    ('normalizer', Normalizer(), ['sex', 'bmi']),
])
column_transformer.fit(features)

ColumnTransformer(transformers=[('min_max_scaler', MinMaxScaler(),
                                 ['age', 'children']),
                                ('standard_scaler', StandardScaler(),
                                 ['smoker', 'region']),
                                ('normalizer', Normalizer(), ['sex', 'bmi'])])

In [20]:
transformed_features = column_transformer.transform(features)

In [21]:
transformed_features[:2]

array([[ 0.02173913,  0.        ,  1.97058663,  1.34390459,  0.        ,
         1.        ],
       [ 0.        ,  0.2       , -0.5074631 ,  0.43849455,  0.02959911,
         0.99956185]])

### Spitting examples

In the following code cell we are going to split features and labels into training, testing and validation sets using the `train_test_split` from `sklearn.model_selection`.

In [22]:
train_features, test_features, train_labels, test_labels = train_test_split(
    transformed_features,
    labels,
    random_state=SEED,
    test_size = .1
)
train_features, val_features, train_labels, val_labels = train_test_split(
    train_features,
    train_labels,
    random_state = SEED,
    test_size = .1
)

### Dataset
The next step is to create a dataset. This process is done so that we can be able to batch our data. Our dataset will be called the `Insuarance` dataset and it takes in features(X) with their respective labels(y).

In [26]:
class Insuarance(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __getitem__(self, index):
        sample = self.X[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        return len(self.y)
    
    def __str__(self):
        return "Insuarance Dataset"
    
    def __repr__(self):
        return "Insuarance Dataset"

### Custom Transform `ToTensor`

This transformation converts features and labels to `torch` tensors.

In [46]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.FloatTensor(inputs), torch.tensor(targets)

### Generating sets from the `Insuarance` dataset.

In [41]:
train = Insuarance(
    train_features,
    train_labels,
    transform= ToTensor()
)

val = Insuarance(
    val_features,
    val_labels,
    transform= ToTensor()
)

test = Insuarance(
    test_features,
    test_labels,
    transform= ToTensor()
)

### Creating iterators

Now we can batch our data into batches of `32`

In [42]:
BATCH_SIZE = 32

train_iterator = DataLoader(train, batch_size= BATCH_SIZE, shuffle=True)
test_iterator = DataLoader(test, batch_size= BATCH_SIZE, shuffle=False)
val_iterator = DataLoader(val, batch_size= BATCH_SIZE, shuffle=False)

### Checking a single train example

In [43]:
train[0]

(tensor([ 0.9565,  0.0000, -0.5075,  1.3439,  0.0267,  0.9996]),
 tensor(12979.3580, dtype=torch.float64))

### Checking the first Batch in the test set

In [44]:
next(iter(test_iterator))

[tensor([[ 0.5870,  0.4000, -0.5075, -1.3723,  0.0000,  1.0000],
         [ 0.3913,  0.0000, -0.5075, -0.4669,  0.0000,  1.0000],
         [ 1.0000,  0.0000,  1.9706, -0.4669,  0.0000,  1.0000],
         [ 0.6087,  0.6000, -0.5075, -0.4669,  0.0388,  0.9992],
         [ 0.0217,  0.0000,  1.9706, -0.4669,  0.0313,  0.9995],
         [ 0.3478,  0.2000, -0.5075,  1.3439,  0.0233,  0.9997],
         [ 0.0217,  0.0000, -0.5075, -0.4669,  0.0000,  1.0000],
         [ 1.0000,  0.0000, -0.5075, -0.4669,  0.0264,  0.9997],
         [ 0.2174,  0.0000, -0.5075, -1.3723,  0.0000,  1.0000],
         [ 0.6739,  0.6000, -0.5075, -0.4669,  0.0348,  0.9994],
         [ 0.2609,  0.6000,  1.9706,  1.3439,  0.0409,  0.9992],
         [ 0.5000,  0.4000, -0.5075, -0.4669,  0.0327,  0.9995],
         [ 0.2391,  0.2000, -0.5075,  0.4385,  0.0000,  1.0000],
         [ 0.6087,  0.6000,  1.9706,  0.4385,  0.0236,  0.9997],
         [ 0.9130,  0.0000,  1.9706,  0.4385,  0.0244,  0.9997],
         [ 0.6304,  0.400

### Regressor Model

This is a simple regression pytorch module for our regression task.

In [50]:
class Regressor(nn.Module):
    def __init__(self, in_features, out_features=1):
        super(Regressor, self).__init__()
        self.fc_1 = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        x = self.fc_1(x)
        return x


In [51]:
in_features = train_features.shape[-1]
out_features = 1

regressor = Regressor(in_features, out_features).to(device)
regressor

Regressor(
  (fc_1): Linear(in_features=6, out_features=1, bias=True)
)

### Optimizer and Criterion
 
For the optimizer we are going to use the `Adam` optimizer for our `back_propagation` algorithim with default parameters. For the loss function since this is a regression task we are goin to use `MSE`(Mean Squared Error) alternatively we can use `MAE`(Mean Absolute Error).

In [83]:
optimizer = torch.optim.Adam(regressor.parameters())
criterion = nn.MSELoss(reduction='mean').to(device)

### Train and Evaluate functions.

In the following code cell we are going to create a `train` and `evaluate` functions. These functions returns us the average loss per epoch.

In [76]:
def train(iterator, model, device, criterion, optimizer):
    epoch_loss = 0.
    model.train()
    for X, y in iterator:
        X = X.to(device)
        y = y.to(device).float()
        model.zero_grad()
        preds = model(X)
        loss = criterion(preds, y.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss
    return epoch_loss/len(iterator)

def evaluate(iterator, model, device, criterion):
    epoch_loss = 0.
    model.eval()
    with torch.no_grad():
        for X, y in iterator:
            X = X.to(device)
            y = y.to(device).float()
            preds = model(X)
            loss = criterion(preds, y.unsqueeze(1))
            epoch_loss += loss
    return epoch_loss/len(iterator)

### Helper Functions

In the following code cell we are going to create helper function that will help us to visualize our training loop.


In [77]:
import time
from prettytable import PrettyTable

In [78]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [79]:
def tabulate_training(column_names, data, title):
    table = PrettyTable(column_names)
    table.title= title
    table.align[column_names[0]] = 'l'
    table.align[column_names[1]] = 'r'
    table.align[column_names[2]] = 'r'
    for row in data:
        table.add_row(row)
    print(table)

### Training loop

In the folloing code cell we are going to run the training loop.

In [100]:
EPOCHS = 5_000

MODEL_NAME = 'best_regression_model.pt'
best_valid_loss = float('inf')
column_names = ["Set", "Loss", "ETA (time)"]

for epoch in range(EPOCHS):
    start = time.time()
    train_loss = train(train_iterator, regressor, device, criterion, optimizer)
    end = time.time()
    eval_loss = evaluate(val_iterator, regressor, device, criterion)

    title = f"EPOCH {epoch+1:02}/{EPOCHS} {'(saving)' if eval_loss < best_valid_loss else '(not saving)'}"
    if eval_loss < best_valid_loss:
        best_valid_loss = eval_loss
        torch.save(regressor.state_dict(), MODEL_NAME)

    data_rows = [
          ["Train", f'{train_loss:.3f}', hms_string(end-start)],
          ["Validation", f'{eval_loss:.3f}', ""]
    ]
    tabulate_training(column_names, data_rows, title)

+-----------------------------------------+
|          EPOCH 01/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322227200.000 | 0:00:00.07 |
| Validation | 232769248.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 02/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 323429696.000 | 0:00:00.05 |
| Validation | 232766672.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 03/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322428192.000 | 0:00:00.05 |
| Validation | 232764064.000 |  

+-----------------------------------------+
|          EPOCH 28/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 321974240.000 | 0:00:00.06 |
| Validation | 232698752.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 29/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322444256.000 | 0:00:00.06 |
| Validation | 232696128.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 30/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322295424.000 | 0:00:00.06 |
| Validation | 232693504.000 |  

+-----------------------------------------+
|          EPOCH 52/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 321627232.000 | 0:00:00.06 |
| Validation | 232636032.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 53/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322619008.000 | 0:00:00.05 |
| Validation | 232633472.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 54/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322332160.000 | 0:00:00.05 |
| Validation | 232630784.000 |  

+-----------------------------------------+
|          EPOCH 77/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322677856.000 | 0:00:00.06 |
| Validation | 232570768.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 78/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322528416.000 | 0:00:00.05 |
| Validation | 232568160.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|          EPOCH 79/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322160480.000 | 0:00:00.05 |
| Validation | 232565536.000 |  

+-----------------------------------------+
|         EPOCH 101/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322129920.000 | 0:00:00.06 |
| Validation | 232508208.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 102/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322746432.000 | 0:00:00.05 |
| Validation | 232505584.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 103/5000 (saving)         |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 322799744.000 | 0:00:00.05 |
| Validation | 232502944.000 |  

...

+-----------------------------------------+
|         EPOCH 4879/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307063488.000 | 0:00:00.07 |
| Validation | 220258256.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4880/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 308830848.000 | 0:00:00.05 |
| Validation | 220255760.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4881/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 308588800.000 | 0:00:00.06 |
| Validation | 220253232.000 |  

+-----------------------------------------+
|         EPOCH 4903/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307838176.000 | 0:00:00.07 |
| Validation | 220197840.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4904/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307142368.000 | 0:00:00.06 |
| Validation | 220195312.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4905/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 308209824.000 | 0:00:00.06 |
| Validation | 220192784.000 |  

+-----------------------------------------+
|         EPOCH 4928/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 308779776.000 | 0:00:00.06 |
| Validation | 220134848.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4929/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307330048.000 | 0:00:00.07 |
| Validation | 220132336.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4930/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 308739840.000 | 0:00:00.06 |
| Validation | 220129840.000 |  

+-----------------------------------------+
|         EPOCH 4953/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307584736.000 | 0:00:00.05 |
| Validation | 220072000.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4954/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307418240.000 | 0:00:00.07 |
| Validation | 220069424.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4955/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 306988032.000 | 0:00:00.05 |
| Validation | 220066944.000 |  

+-----------------------------------------+
|         EPOCH 4980/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307240960.000 | 0:00:00.06 |
| Validation | 220003952.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4981/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307574848.000 | 0:00:00.06 |
| Validation | 220001408.000 |            |
+------------+---------------+------------+
+-----------------------------------------+
|         EPOCH 4982/5000 (saving)        |
+------------+---------------+------------+
| Set        |          Loss | ETA (time) |
+------------+---------------+------------+
| Train      | 307231744.000 | 0:00:00.06 |
| Validation | 219998928.000 |  

### Evaluating the best Model.

In the following code cell we are going to evaluate the best saved model for our regression task.

In [101]:
test_loss = evaluate(test_iterator, regressor, device, criterion)

print(f"Test MSE Loss: {test_loss:.2f}")

Test MSE Loss: 238973488.00


### Model Inference

In the following code cells we are going to make predictions on our `regressor` model.

In [102]:
for X, y in test_iterator:
    break

In [103]:
X.shape, y.shape

(torch.Size([32, 6]), torch.Size([32]))

In [104]:
def predict(X, model):
    model.eval()
    with torch.no_grad():
        return model(X)

In [105]:
predict(X[:3], regressor), y[:3]

(tensor([[ 740.9551],
         [ 566.7127],
         [1387.3540]]),
 tensor([ 9095.0683,  5272.1758, 29330.9832], dtype=torch.float64))